In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q torch torchvision
!pip install -q segmentation-models-pytorch albumentations
!apt-get update -qq && apt-get install -qq aria2

In [ ]:
import os

# Change this to wherever you’d like in your Drive
DRIVE_DATA_DIR = "/content/drive/MyDrive/isic2018"

# Create folders if they don’t exist
os.makedirs(os.path.join(DRIVE_DATA_DIR, "images"), exist_ok=True)
os.makedirs(os.path.join(DRIVE_DATA_DIR, "masks"),  exist_ok=True)

In [ ]:
from zipfile import ZipFile
from pathlib import Path
import subprocess

# URLs
IMG_URL  = "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip"
MSK_URL  = "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip"

# Paths on Drive
zip_img_drive = f"{DRIVE_DATA_DIR}/ISIC2018_Task1-2_Training_Input.zip"
zip_msk_drive = f"{DRIVE_DATA_DIR}/ISIC2018_Task1_Training_GroundTruth.zip"

# Download if missing
if not Path(zip_img_drive).exists():
    subprocess.run(["aria2c", "-x16", "-d", DRIVE_DATA_DIR, IMG_URL], check=True)
if not Path(zip_msk_drive).exists():
    subprocess.run(["aria2c", "-x16", "-d", DRIVE_DATA_DIR, MSK_URL], check=True)

# Unzip if folders empty
img_folder = Path(f"{DRIVE_DATA_DIR}/images")
if not any(img_folder.iterdir()):
    with ZipFile(zip_img_drive, 'r') as z: z.extractall(img_folder)
msk_folder = Path(f"{DRIVE_DATA_DIR}/masks")
if not any(msk_folder.iterdir()):
    with ZipFile(zip_msk_drive, 'r') as z: z.extractall(msk_folder)

In [ ]:
!mkdir -p "/content/drive/MyDrive/Colab Notebooks"
!mv "/content/drive/MyDrive/isic2018" "/content/drive/MyDrive/Colab Notebooks/"


In [ ]:
import os
import pandas as pd

# Base Drive folder
BASE = "/content/drive/MyDrive/Colab Notebooks/isic2018"

# Find the actual image & mask sub-folders automatically:
img_root = os.path.join(BASE, "images")
mask_root = os.path.join(BASE, "masks")

# There should be exactly one directory in each:
img_sub = next(d for d in os.listdir(img_root)  if os.path.isdir(os.path.join(img_root, d)))
msk_sub = next(d for d in os.listdir(mask_root) if os.path.isdir(os.path.join(mask_root, d)))

IMG_DIR = os.path.join(img_root,  img_sub)
MSK_DIR = os.path.join(mask_root, msk_sub)

print("Images in:", IMG_DIR)
print("Masks in: ", MSK_DIR)

# Now collect IDs
ids = [os.path.splitext(f)[0] for f in os.listdir(IMG_DIR) if f.lower().endswith(".jpg")]

# Build DataFrame
df = pd.DataFrame({
    "id":         ids,
    "image_path": [os.path.join(IMG_DIR, id + ".jpg")            for id in ids],
    "mask_path":  [os.path.join(MSK_DIR, id + "_segmentation.png") for id in ids]
})

# Quick existence sanity check
print("\nExistence check (first 5):")
for p in df.image_path[:5]:
    print(p, "→", os.path.exists(p))
for p in df.mask_path[:5]:
    print(p, "→", os.path.exists(p))

df.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

print(f"Training samples:   {len(train_df)}")
print(f"Validation samples: {len(val_df)}")

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from PIL import Image

# Define transforms
train_transforms = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.Normalize(),          # mean=0.0, std=1.0 by default
    ToTensorV2()
])
val_transforms = A.Compose([
    A.Resize(256, 256),
    A.Normalize(),
    ToTensorV2()
])

class SkinSegDataset(Dataset):
    def __init__(self, df, transforms):
        self.df = df.reset_index(drop=True)
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Load image and mask
        img = np.array(Image.open(row.image_path).convert("RGB"))
        msk = np.array(Image.open(row.mask_path).convert("L")) // 255  # binary 0/1

        # Apply transforms
        augmented = self.transforms(image=img, mask=msk)
        image = augmented["image"]
        mask  = augmented["mask"].unsqueeze(0).float()  # add channel dim

        return image, mask

# Create datasets
train_ds = SkinSegDataset(train_df, train_transforms)
val_ds   = SkinSegDataset(val_df,   val_transforms)

# Create loaders
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True,  num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=8, shuffle=False, num_workers=2, pin_memory=True)

# Sanity check a batch
images, masks = next(iter(train_loader))
print("Batch image shape:", images.shape)  # e.g. [8, 3, 256, 256]
print("Batch mask  shape:", masks.shape)   # e.g. [8, 1, 256, 256]

In [ ]:
import torch
import segmentation_models_pytorch as smp

# Choose device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate U‑Net
model = smp.Unet(
    encoder_name="resnet34",        # choose encoder
    encoder_weights="imagenet",     # use ImageNet pretraining
    in_channels=3,                  # input RGB images
    classes=1,                      # output binary mask
)

model = model.to(device)
print(model)

In [ ]:
import torch.nn as nn
import torch

# Loss functions
bce_loss  = nn.BCEWithLogitsLoss()
dice_loss = smp.losses.DiceLoss(mode="binary")

def loss_fn(preds, targets):
    return bce_loss(preds, targets) + dice_loss(preds, targets)

# IoU metric
def iou_score(preds, targets, thresh=0.5):
    preds_bin = (torch.sigmoid(preds) > thresh).float()
    inter = (preds_bin * targets).sum(dim=(1,2,3))
    union = preds_bin.sum(dim=(1,2,3)) + targets.sum(dim=(1,2,3)) - inter
    return ((inter + 1e-6) / (union + 1e-6)).mean()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
from tqdm import tqdm
import torch
import os

best_iou = 0.0
num_epochs = 10

train_losses, val_losses, val_ious = [], [], []

for epoch in range(1, num_epochs+1):
    # ——— Training ———
    model.train()
    train_loss = 0.0
    for imgs, masks in tqdm(train_loader, desc=f"Epoch {epoch} [Train]"):
        imgs, masks = imgs.to(device), masks.to(device)
        optimizer.zero_grad()
        preds = model(imgs)
        loss = loss_fn(preds, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    # ——— Validation ———
    model.eval()
    val_loss, val_iou = 0.0, 0.0
    with torch.no_grad():
        for imgs, masks in tqdm(val_loader, desc=f"Epoch {epoch} [Val]"):
            imgs, masks = imgs.to(device), masks.to(device)
            preds = model(imgs)
            val_loss += loss_fn(preds, masks).item()
            val_iou  += iou_score(preds, masks).item()
    val_loss /= len(val_loader)
    val_iou  /= len(val_loader)

    # Checkpoint best
    if val_iou > best_iou:
        best_iou = val_iou
        torch.save(model.state_dict(),
                   "/content/drive/MyDrive/Colab Notebooks/isic2018/best_unet.pth")

    # Log & record
    print(f"\nEpoch {epoch:02d} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss:   {val_loss:.4f} | "
          f"Val IoU:    {val_iou:.4f} | "
          f"Best IoU:   {best_iou:.4f}\n")
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_ious.append(val_iou)

In [ ]:
import matplotlib.pyplot as plt
import os

os.makedirs("outputs/curves", exist_ok=True)

# Loss curve
plt.figure(figsize=(6,4))
plt.plot(range(1, num_epochs+1), train_losses, label="Train Loss")
plt.plot(range(1, num_epochs+1), val_losses,   label="Val Loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend()
plt.title("Training & Validation Loss")
plt.savefig("outputs/curves/loss_curve.png", bbox_inches="tight")
plt.close()

# IoU curve
plt.figure(figsize=(6,4))
plt.plot(range(1, num_epochs+1), val_ious, label="Val IoU")
plt.xlabel("Epoch"); plt.ylabel("IoU"); plt.legend()
plt.title("Validation IoU Over Epochs")
plt.savefig("outputs/curves/iou_curve.png", bbox_inches="tight")
plt.close()

In [ ]:
import numpy as np
from PIL import Image

# Load best model weights
model.load_state_dict(torch.load(
    "/content/drive/MyDrive/Colab Notebooks/isic2018/best_unet.pth",
    map_location=device
))
model.eval()

# Grab one batch
imgs, masks = next(iter(val_loader))
imgs, masks = imgs.to(device), masks.to(device)

with torch.no_grad():
    preds = torch.sigmoid(model(imgs)) > 0.5

os.makedirs("outputs/predictions", exist_ok=True)

# Save first 3 prediction triplets
for i in range(3):
    fig, axes = plt.subplots(1,3, figsize=(12,4))
    axes[0].imshow(imgs[i].cpu().permute(1,2,0));  axes[0].axis("off")
    axes[1].imshow(masks[i,0].cpu(), cmap="gray"); axes[1].axis("off")
    axes[2].imshow(preds[i,0].cpu(), cmap="gray"); axes[2].axis("off")
    plt.tight_layout()

    fname = f"outputs/predictions/sample_{i}.png"
    fig.savefig(fname, bbox_inches="tight")
    plt.close(fig)
    # (optional) display inline
    display(Image.open(fname))

In [ ]:
os.makedirs("outputs/sample_pairs", exist_ok=True)

for idx in [0, 50, 100]:
    row = df.iloc[idx]
    img = np.array(Image.open(row.image_path).convert("RGB"))
    msk = np.array(Image.open(row.mask_path).convert("L"))

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,4))
    ax1.imshow(img); ax1.set_title(f"Image [{row.id}]"); ax1.axis("off")
    ax2.imshow(msk, cmap="gray"); ax2.set_title(f"Mask  [{row.id}]"); ax2.axis("off")

    fname = f"outputs/sample_pairs/{row.id}.png"
    fig.savefig(fname, bbox_inches="tight")
    plt.close(fig)

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/isic2018/best_unet.pth" /content/
from google.colab import files
files.download("best_unet.pth")